## Data Cleaning for Lead Exposure Project

In [1]:
# Packages to import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Import data file 

# Documentation for the data set is located at https://www.franklincountyauditor.com/AUDR-website/media/Documents/FTP/Parcel-CSV-description-of-fields.pdf
url = "https://apps.franklincountyauditor.com/Parcel_CSV/2025/05/Parcel.csv"
df = pd.read_csv(url)

/var/folders/2l/qpy8_v3d5x7glst5y2hhkz6m0000gn/T/ipykernel_22461/1073287580.py:3: DtypeWarning: Columns (12,13,14,15,31,33,54,59,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [28]:
df

,PARCEL ID,AEXMLND,AEXMBLD,AEXMTOT,APPRLND,APPRBLD,APPRTOT,AUDMAP,AUDRTG,LANDUSE,...,NOSTORY,YEARBLT,PROPTYP,WALL,TIFMLND,TIFMBLD,POINT_X,POINT_Y,HOMSTD,BANKCODE
0,010-000001,0.0,0.0,0.0,11300.0,73300.0,84600.0,H001,71.0,520.0,...,20.0,1900.0,NaN,STUCCO,203700.0,118700.0,1.828201e+06,721773.867345,NaN,NaN
1,010-000002,0.0,0.0,0.0,121800.0,160000.0,281800.0,R044,94.0,510.0,...,15.0,1910.0,NaN,WD/ALUM/VINYL,0.0,0.0,1.823726e+06,738685.399302,NaN,NaN
2,010-000003,0.0,0.0,0.0,37800.0,4500.0,42300.0,K037,191.0,399.0,...,NaN,NaN,NaN,NaN,0.0,0.0,1.835470e+06,705365.735381,NaN,NaN
3,010-000004,0.0,297900.0,297900.0,114900.0,0.0,114900.0,I017,80.0,510.0,...,20.0,2017.0,NaN,WD/ALUM/VINYL,0.0,0.0,1.840164e+06,714787.198657,NaN,NaN
4,010-000005,0.0,0.0,0.0,100000.0,403200.0,503200.0,N047,172.0,401.0,...,NaN,NaN,NaN,NaN,0.0,0.0,1.813787e+06,725871.027785,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483882,610-341317,0.0,0.0,0.0,0.0,0.0,0.0,T106J,92.0,NaN,...,10.0,2016.0,NaN,WD/ALUM/VINYL,0.0,0.0,1.826366e+06,771805.490542,NaN,NaN
483883,610-341318,0.0,0.0,0.0,0.0,0.0,0.0,T106J,93.0,NaN,...,10.0,2016.0,NaN,WD/ALUM/VINYL,0.0,0.0,1.826366e+06,771805.490542,NaN,NaN
483884,610-341319,0.0,0.0,0.0,0.0,0.0,0.0,T106J,94.0,NaN,...,10.0,2016.0,NaN,WD/ALUM/VINYL,0.0,0.0,1.826366e+06,771805.490542,NaN,NaN
483885,610-341320,0.0,0.0,0.0,0.0,0.0,0.0,T106J,95.0,NaN,...,10.0,2016.0,NaN,3/6 MASONRY TO FRAME .500,0.0,0.0,1.826366e+06,771805.490542,NaN,NaN


In [54]:
df["PCLASS"].head()

columns_to_keep = ['PARCEL ID', 'APPRTOT', 'PRICE', 'PCLASS', 'GRADE', 'YEARBLT', 'POINT_X', 'POINT_Y']

df_column_filtered = df[columns_to_keep]

df_column_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483887 entries, 0 to 483886
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   PARCEL ID  483887 non-null  object 
 1   APPRTOT    483887 non-null  float64
 2   PRICE      482896 non-null  float64
 3   PCLASS     469952 non-null  object 
 4   GRADE      406804 non-null  object 
 5   YEARBLT    406791 non-null  float64
 6   POINT_X    483887 non-null  float64
 7   POINT_Y    483860 non-null  object 
dtypes: float64(4), object(4)
memory usage: 29.5+ MB


In [60]:
df_column_filtered['POINT_Y'].head()

0    721773.867345
1    738685.399302
2    705365.735381
3    714787.198657
4    725871.027785
Name: POINT_Y, dtype: object

In [64]:
df_column_filtered = df_column_filtered.astype({'PCLASS':'str', 'GRADE':'str', 'POINT_Y':'float64'})

ValueError: could not convert string to float: 'Y': Error while type casting for column 'POINT_Y'

In [56]:
df_filtered = df_column_filtered[df_column_filtered['PCLASS'].isin(['R'])]

In [57]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422709 entries, 0 to 483886
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   PARCEL ID  422709 non-null  object 
 1   APPRTOT    422709 non-null  float64
 2   PRICE      422066 non-null  float64
 3   PCLASS     422709 non-null  object 
 4   GRADE      401208 non-null  object 
 5   YEARBLT    401194 non-null  float64
 6   POINT_X    422709 non-null  float64
 7   POINT_Y    422684 non-null  object 
dtypes: float64(4), object(4)
memory usage: 29.0+ MB
